### Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
from astropy.constants import si as constants
from astropy.table import Table
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm 
from tabulate import tabulate
from survey_tools import sky
from survey_tools.utility import plot
import targets

### Parameters

In [31]:
instrument        = 'GNIRS'
resolving_power   = 7200
wavelength_ranges = np.array([[11700, 13700],[14900, 18000],[19100,24900]]) # angstrom
location          = 'MaunaKea' # MaunaKea, Paranal
min_ngs_mag       =  8.0
max_ngs_mag       = 18.0
max_ngs_sep       = 25.0
wfs_band          = 'R'
obs_epoch         = 2025.0

airmass           = 1.5
min_photon_rate   = 10   # ph/s/arcsec^2/nm/m^2
trans_minimum     = 0.85 # percent
trans_multiple    = 0.5  # FWHM of Ha
avoid_multiple    = 0.5  # FWHM of Ha

default_dz_z = 3e-4
default_radius = 1.0/2 # arcsec
default_flux_Ha = 15e-17 # erg/s/cm^2
default_velocity_dispersion = 200 # km/s
default_flux_NII_Ha_ratio = 0.10

In [32]:
# From: Kalita et al (2025) arXiv:2501.03328v1
galaxies = Table(rows=[
    (1987, 149.721750, 1.965500, 1.626438, 10.97, 0.04, 10.25, 0.21, 2.15, 0.22, 10.95, 0.20, 2.15, 0.22,  0,  4),
    ( 147, 149.745833, 2.125944, 1.556338, 11.07, 0.02, 10.07, 0.18, 2.00, 0.20, 10.88, 0.17, 2.00, 0.20,  3,  5),
    (1704, 149.775542, 2.251639, 1.673098, 10.90, 0.04,  9.73, 0.17, 1.83, 0.20, 10.65, 0.16, 1.83, 0.20,  0,  1),
    (2423, 149.825125, 1.978889, 1.487085, 10.91, 0.03, 10.60, 0.17, 1.85, 0.22, 10.45, 0.16, 1.85, 0.22,  1,  8),
    (1943, 149.832750, 2.024306, 1.482637, 10.90, 0.02, 10.06, 0.20, 1.76, 0.17, 10.82, 0.11, 1.76, 0.17,  1,  3),
    (1147, 149.854708, 2.115111, 1.481732, 11.05, 0.08, 10.34, 0.17, 2.01, 0.24, 10.74, 0.20, 2.01, 0.24,  0,  2),
    (  89, 149.855750, 2.130222, 1.478401, 11.09, 0.03, 10.27, 0.13, 1.74, 0.23, 10.55, 0.18, 1.74, 0.23,  2,  2),
    (1831, 149.877875, 2.297639, 1.509195, 10.96, 0.05, 10.32, 0.15, 1.90, 0.21, 10.71, 0.16, 1.90, 0.21,  0,  3),
    (1766, 149.887792, 2.499833, 1.672921, 10.92, 0.02, 10.36, 0.20, 2.01, 0.17, 10.87, 0.13, 2.01, 0.17,  3,  3),
    ( 682, 149.889708, 2.358778, 1.501451, 10.79, 0.02,  9.96, 0.16, 1.63, 0.20, 10.61, 0.14, 1.63, 0.20,  2,  6),
    ( 126, 149.912250, 2.281639, 1.550443, 10.96, 0.02, 10.40, 0.20, 1.65, 0.16, 10.64, 0.15, 1.65, 0.16,  7,  8),
    (1928, 149.923208, 1.898194, 1.552804, 11.02, 0.02,  9.91, 0.18, 1.89, 0.21, 10.81, 0.16, 1.89, 0.21,  0,  2),
    ( 193, 149.925333, 2.135028, 1.470131, 10.91, 0.05, 10.14, 0.17, 1.85, 0.29, 10.66, 0.18, 1.85, 0.29,  2,  6),
    (1945, 149.938375, 1.950056, 1.444649, 10.88, 0.05,  9.98, 0.23, 1.19, 0.22, 10.15, 0.15, 1.19, 0.22,  1,  3),
    ( 285, 149.972667, 2.490139, 1.453462, 10.92, 0.02,  9.93, 0.18, 1.90, 0.24, 10.65, 0.19, 1.90, 0.24,  0,  1),
    (3074, 149.985375, 2.561778, 1.557741, 10.96, 0.08, 10.23, 0.12, 1.90, 0.22, 10.80, 0.16, 1.90, 0.22,  1,  2),
    (2074, 149.997875, 2.480861, 1.462140, 11.05, 0.02, 10.22, 0.17, 2.07, 0.23, 10.90, 0.17, 2.07, 0.23, 14, 15),
    (1861, 150.001083, 2.321444, 1.459471, 11.00, 0.02, 10.34, 0.17, 1.69, 0.21, 10.59, 0.15, 1.69, 0.21,  2,  3),
    (1749, 150.025000, 2.355278, 1.610989, 10.87, 0.06, 10.30, 0.20, 1.72, 0.23, 10.53, 0.18, 1.72, 0.23,  2,  3),
    ( 281, 150.124042, 2.447583, 1.599120, 10.97, 0.03, 10.61, 0.16, 1.78, 0.22, 10.59, 0.18, 1.78, 0.22,  3,  3),
    (  81, 150.164458, 1.936306, 1.525303, 10.94, 0.02, 10.09, 0.19, 1.84, 0.21, 10.65, 0.16, 1.84, 0.21,  7,  8),
    (2151, 150.200875, 2.460639, 1.582532, 10.79, 0.04, 10.22, 0.20, 1.70, 0.23, 10.63, 0.16, 1.70, 0.23,  4,  7),
    (1663, 150.220708, 2.013139, 1.604375, 10.86, 0.04, 10.11, 0.17, 1.99, 0.23, 10.81, 0.17, 1.99, 0.23,  4,  4),
    ( 482, 150.258083, 2.240167, 1.603783, 10.89, 0.02, 10.15, 0.13, 1.88, 0.22, 10.72, 0.17, 1.88, 0.22,  7,  7),
    ( 428, 150.292500, 2.422167, 1.635788, 11.05, 0.04, 10.19, 0.15, 2.19, 0.22, 10.96, 0.19, 2.19, 0.22,  7, 11),
    (2905, 150.302250, 1.931889, 1.548028, 10.94, 0.05,  9.89, 0.17, 1.86, 0.19, 10.87, 0.14, 1.86, 0.19,  4,  7),
    (1394, 150.365458, 2.227500, 1.699351, 11.28, 0.05, 10.38, 0.17, 1.99, 0.14, 11.02, 0.11, 1.99, 0.14,  3,  3),
    (3165, 150.394917, 2.456361, 1.438470, 10.74, 0.04, 10.12, 0.17, 1.91, 0.24, 10.65, 0.19, 1.91, 0.24,  6,  9),
    (1334, 150.402917, 2.408833, 1.514096, 10.54, 0.04,  9.93, 0.16, 1.88, 0.20, 10.79, 0.18, 1.88, 0.20,  3,  7),
    ( 852, 150.441542, 2.129222, 1.557442, 11.44, 0.02, 10.44, 0.12, 2.27, 0.23, 10.98, 0.15, 2.27, 0.23,  6, 10),
    (4003, 150.469292, 2.476528, 1.579673, 10.86, 0.10, 10.00, 0.15, 1.69, 0.22, 10.62, 0.16, 1.69, 0.22,  4,  4),
    ( 495, 150.482708, 2.304139, 1.485021, 10.66, 0.02,  9.68, 0.18, 1.90, 0.23, 10.59, 0.18, 1.90, 0.23,  3,  7),
], names=[
    'id', 'ra', 'dec', 'z',
    'mass', 'mass_unc',
    'mass_buldge', 'mass_buldge_unc',
    'sfr_buldge', 'sfr_buldge_unc',
    'mass_disk', 'mass_disk_unc',
    'sfr_disk', 'sfr_disk_unc',
    'clumps', 'clump_candidates'
])

In [33]:
if 'z_unc' not in galaxies.colnames:
    galaxies['z_unc'] = default_dz_z * (1 + galaxies['z'])
if 'radius' not in galaxies.colnames:
    galaxies['radius'] = default_radius
if 'flux_Ha' not in galaxies.colnames:
    galaxies['flux_Ha'] = default_flux_Ha
if 'flux_NIIa' not in galaxies.colnames:
    galaxies['flux_NIIa'] = default_flux_Ha * default_flux_NII_Ha_ratio
if 'flux_NIIb' not in galaxies.colnames:
    galaxies['flux_NIIb'] = default_flux_Ha * default_flux_NII_Ha_ratio
if 'velocity_dispersion' not in galaxies.colnames:
    galaxies['velocity_dispersion'] = default_velocity_dispersion

In [34]:
if np.ndim(wavelength_ranges) == 1:
    min_wavelength = wavelength_ranges[0]
    max_wavelength = wavelength_ranges[1]
else:
    min_wavelength = wavelength_ranges[0,0]
    max_wavelength = wavelength_ranges[-1,1]

sky_transmission_data = sky.load_transmission_data(location, airmass)
sky_background_data = sky.load_background_data(location, airmass)

### Gather Emission Line Information

In [35]:
eml_names = np.array(['Ha', 'NIIa', 'NIIb'])
eml_wavelength_rest = np.array([6564.610, 6549.89, 6585.27])
eml_wavelength_vac = np.zeros((len(galaxies), len(eml_names)))
eml_wavelength_atm = np.zeros((len(galaxies), len(eml_names)))
eml_transmission   = np.zeros((len(galaxies), len(eml_names)))
eml_fwhm           = np.zeros((len(galaxies), len(eml_names)))
eml_dwavelength    = np.zeros((len(galaxies), len(eml_names)))
eml_sigma          = np.zeros((len(galaxies), len(eml_names)))
eml_flux           = np.zeros((len(galaxies), len(eml_names)))
eml_sb             = np.zeros((len(galaxies), len(eml_names)))
eml_ph_energy      = np.zeros((len(galaxies), len(eml_names)))
eml_ph_rate        = np.zeros((len(galaxies), len(eml_names)))
eml_sky_rate       = np.zeros((len(galaxies), len(eml_names)))
eml_reject         = np.zeros((len(galaxies), len(eml_names)), dtype=np.bool)

for i in range(len(eml_names)):
    eml_wavelength_vac[:,i] = eml_wavelength_rest[i] * (1 + galaxies['z'])
    eml_wavelength_atm[:,i] = sky.get_vacuum_to_air_wavelength(eml_wavelength_vac[:,i])
    eml_transmission[:,i] = sky.get_mean_transmission(sky_transmission_data, eml_wavelength_atm[:,i], default_velocity_dispersion, resolving_power, trans_multiple)
    eml_fwhm[:,i] = eml_wavelength_atm[:,i] * default_velocity_dispersion / constants.c.to('km/s').value
    eml_dwavelength[:,i] = np.sqrt((eml_wavelength_atm[:,i]/resolving_power)**2 + eml_fwhm[:,i]**2)
    eml_sigma[:,i] = eml_dwavelength[:,i] / 2.35482 # FWHM -> sigma
    eml_flux[:,i] = galaxies[f"flux_{eml_names[i]}"]
    eml_sb[:,i] = eml_flux[:,i]/(np.pi*galaxies['radius']**2) # erg/s/cm^2/arcsec^2
    eml_ph_energy[:,i] = 6.626e-27 * 2.998e10 / (eml_wavelength_atm[:,i]/1e8) # erg
    eml_ph_rate[:,i] = eml_sb[:,i] / eml_ph_energy[:,i] / (eml_fwhm[:,i]/10) * 100**2  # ph/s/arcsec^2/nm/m^2
    eml_sky_rate[:,i] = sky.get_background(sky_background_data, eml_wavelength_atm[:,i], [eml_wavelength_atm[:,i] - eml_fwhm[:,i]*10, eml_wavelength_atm[:,i] + eml_fwhm[:,i]*10], resolving_power)
    eml_reject[:,i] = sky.reject_emission_line(sky_background_data, sky_transmission_data, eml_wavelength_atm[:,i], default_velocity_dispersion, resolving_power, wavelength_ranges, trans_minimum, trans_multiple, avoid_multiple, min_photon_rate)

for i in range(len(eml_names)):
    galaxies[f"trans_{eml_names[i]}"] = eml_transmission[:,i]
    galaxies[f"reject_{eml_names[i]}"] = eml_reject[:,i]

index_Ha = np.where(eml_names == 'Ha')[0][0]
index_NIIa = np.where(eml_names == 'NIIa')[0][0]
index_NIIb= np.where(eml_names == 'NIIb')[0][0]

min_wavelength_range = (eml_wavelength_atm[:,index_NIIb] - eml_wavelength_atm[:,index_Ha]) * 1.2

del i

### Gather NGS Information

In [36]:
galaxies['ngs_count'] = np.zeros(len(galaxies))
galaxies['ngs_best_sep'] = np.full(len(galaxies), np.nan)
galaxies['ngs_best_mag'] = np.full(len(galaxies), np.nan)

for i in range(len(galaxies)):
    stars = targets.find_nearby_stars(galaxies['ra'][i], galaxies['dec'][i], min_mag=min_ngs_mag, max_mag=max_ngs_mag, max_sep=max_ngs_sep, wfs_band=wfs_band, epoch=obs_epoch)
    galaxies['ngs_count'][i] = len(stars)
    if len(stars) > 0:
        galaxies['ngs_best_sep'][i] = np.min(stars['sep'])
        galaxies['ngs_best_mag'][i] = np.min(stars[f"gaia_{wfs_band}"])

del i

### Export Target Data

In [37]:
galaxies.write(f"../output/{instrument}-galaxies.txt", format='ascii', overwrite=True)

### Details on Selected Target

In [38]:
selected_target_id = 682
target_index = np.where(galaxies['id'] == selected_target_id)[0][0]

In [ ]:
print(f"  Target ID: {galaxies['id'][target_index]}")
print(f"   Redshift: {galaxies['z'][target_index]:.4f}")
print(f"  Ha in Vac: {eml_wavelength_vac[target_index, index_Ha]/10:.2f} nm")
print(f"        Atm: {eml_wavelength_atm[target_index, index_Ha]/10:.2f} nm")
print(f"       FWHM: {eml_fwhm[target_index, index_Ha]/10:.2f} nm [{default_velocity_dispersion:.0f} km/s]")
print(f"          R: {resolving_power:.0f} [dLambda = {eml_dwavelength[target_index, index_Ha]/10:.1f} nm]")
print(f"  Line Flux: {galaxies['flux_Ha'][target_index]:.1e} erg/s/cm^2")
print(f"         SB: {eml_sb[target_index, index_Ha]:.1e} erg/s/cm^2/arcsec^2  [{eml_sb[target_index, index_Ha]/1000:.1e} W/m^2/arcsec^2    ]")
print(f"    Ph Rate: {eml_ph_rate[target_index, index_Ha]:.1f}     ph/s/arcsec^2/nm/m^2 [{eml_ph_rate[target_index, index_Ha]*eml_ph_energy[target_index, index_Ha]*1e-7*1e9:.1e} J/s/m^2/m/arcsec^2]")
print(f"Sky Ph Rate: {eml_sky_rate[target_index, index_Ha]:.1f}     ph/s/arcsec^2/nm/m^2 [{eml_sky_rate[target_index, index_Ha]*eml_ph_energy[target_index, index_Ha]*1e-7*1e9:.1e} J/s/m^2/m/arcsec^2]")

### Sky Transmission Near Selected Target

In [ ]:
plot_dwavelength = max(min_wavelength_range[target_index], eml_dwavelength[target_index,index_Ha]*8)
plot_xrange = np.round(np.array([eml_wavelength_atm[target_index,index_Ha] - plot_dwavelength, eml_wavelength_atm[target_index,index_Ha] + plot_dwavelength])/10, 0)*10
plot_yrange = [70, 101]

wavelength_filter = (sky_transmission_data['wavelength'] >= plot_xrange[0]) & (sky_transmission_data['wavelength'] <= plot_xrange[1])
sky_wavelengths = sky_transmission_data['wavelength'][wavelength_filter]

_, ax = plot.create_plot(title=f"{location} IR Sky Transmission")
ax.plot(sky_wavelengths, sky_transmission_data['transmission'][wavelength_filter]*100, linestyle='-', color='b', linewidth=1)
ax.set_xlabel('Wavelength [Angstrom]')
ax.set_ylabel('Transmission [%]')
ax.set_ylim(plot_yrange)

for i in range(len(eml_names)):
    wavelength_unc = eml_wavelength_atm[target_index,i] * galaxies['z_unc'][target_index]
    color = 'r' if eml_transmission[target_index,i] < trans_minimum else 'g'
    plt.axvline(x=eml_wavelength_atm[target_index,i]-wavelength_unc, linestyle='-', linewidth=0.5, color='k')
    ax.fill_between(sky_wavelengths, plot_yrange[0], plot_yrange[1], where=(abs(sky_wavelengths - eml_wavelength_atm[target_index,i]) < wavelength_unc), facecolor=[0.8,0.8,0.8], alpha=0.3)
    plt.axvline(x=eml_wavelength_atm[target_index,i]+wavelength_unc, linestyle='-', linewidth=0.5, color='k')
    ax.fill_between(sky_wavelengths, plot_yrange[0], plot_yrange[1], where=(abs(sky_wavelengths - eml_wavelength_atm[target_index,i]) < eml_dwavelength[target_index,i]*trans_multiple), facecolor=[0.5,0.5,0.5], alpha=0.3)
    plt.axvline(x=eml_wavelength_atm[target_index,i], linestyle=':', linewidth=1, color='k')
    plt.hlines(y=eml_transmission[target_index,i]*100, xmin=(eml_wavelength_atm[target_index,i] - eml_dwavelength[target_index,i]*trans_multiple), xmax=(eml_wavelength_atm[target_index,i] + eml_dwavelength[target_index,i]*trans_multiple), linestyle='-', linewidth=2, color=color)
    plt.text(0.05, 0.07, f"id={galaxies['id'][target_index]}\nz={galaxies['z'][target_index]:.4f}\ndz={galaxies['z_unc'][target_index]:.4f}", transform=ax.transAxes, bbox=dict(facecolor='white', alpha=0.5))

    print(f"Sky Transmission at {eml_wavelength_atm[target_index,i]/10:.2f} nm ({eml_names[i]}): {eml_transmission[target_index,i]*100:.1f}%")

plt.show()

# Legend:
# - Dotted line is expected wavelength based on redshift
# - Shadded light gray is uncertainty of wavelength based on redshift uncertainty
# - Shadded dark gray is region over which transmision is calculated
# - Horizontal lines are average transmission (green means averge above threshold while red means below)

del plot_dwavelength, plot_xrange, plot_yrange, wavelength_filter, sky_wavelengths, ax, i, color

### Sky Background Near Selected Target

In [ ]:
plot_dwavelength = max(min_wavelength_range[target_index], eml_dwavelength[target_index,index_Ha]*5)
plot_xrange = np.round(np.array([eml_wavelength_atm[target_index, index_Ha] - plot_dwavelength, eml_wavelength_atm[target_index, index_Ha] + plot_dwavelength])/10, 0)*10
plot_yrange = [0.1, 1e4]

sky_wavelengths = np.linspace(plot_xrange[0], plot_xrange[1], 1000)
sky_background_data_low_res = sky.get_low_res_background(sky_background_data, plot_xrange, resolving_power)
sky_lines = sky.find_sky_lines(sky_background_data_low_res, min_photon_rate)

_, ax = plot.create_plot(title=f"{location} IR Sky Background")
ax.plot(sky_background_data_low_res['wavelength'], sky_background_data_low_res['emission'], linestyle='-', color='b', linewidth=1)
ax.scatter(sky_lines['wavelength'], sky_lines['emission'], marker='x', color='b')
plt.hlines(y=sky_lines["width_height"], xmin=sky_lines["wavelength_low"], xmax=sky_lines["wavelength_high"], linestyle='-', color='b')        
ax.axhline(min_photon_rate, linestyle=':', linewidth=1, color='k')
ax.set_xlim(plot_xrange)
ax.set_ylim(plot_yrange)
ax.set_yscale('log')
ax.set_xlabel('Wavelength [Angstrom]')
ax.set_ylabel('Emission [$ph/s/arcsec^2/nm/m^2$]')

for i in range(len(eml_names)):
    wavelength_unc = eml_wavelength_atm[target_index,i] * galaxies['z_unc'][target_index]
    color = 'r' if eml_reject[target_index,i] else 'g'
    plt.axvline(x=eml_wavelength_atm[target_index,i]-wavelength_unc, linestyle='-', linewidth=0.5, color='k')
    ax.fill_between(sky_wavelengths, plot_yrange[0], plot_yrange[1], where=(abs(sky_wavelengths - eml_wavelength_atm[target_index,i]) < wavelength_unc), facecolor=[0.8,0.8,0.8], alpha=0.3)
    plt.axvline(x=eml_wavelength_atm[target_index,i]+wavelength_unc, linestyle='-', linewidth=0.5, color='k')
    ax.fill_between(sky_background_data_low_res['wavelength'], plot_yrange[0], plot_yrange[1], where=(abs(sky_background_data_low_res['wavelength'] - eml_wavelength_atm[target_index,i]) < eml_dwavelength[target_index,i]*avoid_multiple), facecolor=[0.5,0.5,0.5], alpha=0.3)
    ax.plot(sky_wavelengths, eml_ph_rate[target_index,i] * np.sqrt(2*np.pi) * eml_sigma[target_index,i] * norm.pdf(sky_wavelengths, eml_wavelength_atm[target_index,i], eml_sigma[target_index,i]), linestyle='-', linewidth=2, color=color)
    plt.axvline(x=eml_wavelength_atm[target_index,i], linestyle=':', linewidth=1, color='k')
    plt.text(0.05, 0.82, f"id={galaxies['id'][target_index]}\nz={galaxies['z'][target_index]:.4f}\ndz={galaxies['z_unc'][target_index]:.4f}", transform=ax.transAxes, bbox=dict(facecolor='white', alpha=0.5))

    index = np.argmin(np.abs(sky_background_data_low_res['wavelength'] - eml_wavelength_atm[target_index,i]))
    print(f"Sky Background at {sky_background_data_low_res['wavelength'][index]/10:.2f} nm ({eml_names[i]}) = {sky_background_data_low_res['emission'][index]:.1f} ph/s/arcsec^2/nm/m^2")

plt.show()

# Legend:
# - Dotted line is expected wavelength based on redshift
# - Shadded light gray is uncertainty of wavelength based on redshift uncertainty
# - Shadded dark gray is sky line avoidance region
# - Green profile means no line in avoidance region
# - Red profile means there is a sky line in avoidance region

del plot_dwavelength, plot_xrange, plot_yrange, ax, i, color, index

### Stars Near Selected Target

In [ ]:
stars = targets.find_nearby_stars(galaxies['ra'][target_index], galaxies['dec'][target_index], min_mag=min_ngs_mag, max_mag=max_ngs_mag, max_sep=max_ngs_sep, wfs_band=wfs_band, epoch=obs_epoch)

if len(stars) == 0:
    print("No stars found")
else:
    warnings.simplefilter('ignore', UserWarning)
    display(tabulate(stars, headers=stars.colnames, tablefmt='html'))
    warnings.resetwarnings()